In [6]:
%load_ext autoreload
%autoreload 2
%load_ext autotime

import os
import sys
sys.path.append('..')

import splintr as sp
from splintr.splice import rmats_subset_top_events
# sp.util.verbose = True

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torchsummary import summary

from ax.plot.contour import plot_contour, interact_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting
init_notebook_plotting()

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

seed = 99
np.random.seed(seed)
torch.manual_seed(seed)
torch.set_num_threads=16

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


[INFO 08-07 22:28:03] ipy_plotting: Injecting Plotly library into cell. Do not overwrite or delete cell.


time: 7.37 ms


In [2]:
# Parameters
data_dir = '../data/features'
feature_file = f'{data_dir}/SE.txt'
feature_df = rmats_subset_top_events(feature_file, 5)
feature_df = feature_df.loc[feature_df.IncLevelDifference > 0] # upregulated AS events

time: 93.1 ms


In [3]:
datasets, samplers, label_names = sp.learn.load_datasets(feature_df, '../data/hg19.fa', seq_size=384, augment_k=5)
train_loader, valid_loader, test_loader = sp.learn.init_loaders(datasets, samplers, batch_size=32)


time: 8.98 s


In [49]:
def train_evaluate(params):
    train_loader, valid_loader, test_loader = sp.learn.init_loaders(datasets, samplers, batch_size=params['batch_size'])
    net, valid_acc = sp.learn.fit(params, train_loader, valid_loader, num_epochs=30, label_names=label_names, log_dir='/home/ubuntu/tb/8-07-19-6class/')
    return valid_acc

time: 1.76 ms


In [ ]:
best_params, values, experiment, model = optimize(
    parameters=[
        {"name": "batch_size", "type": "range", "bounds": [10, 1000], "log_scale": True},
        {"name": "filter1", "type": "range", "bounds": [4, 256], "log_scale": True},
        {"name": "filter2", "type": "range", "bounds": [4, 256], "log_scale": True},
        {"name": "main_kernel", "type": "range", "bounds": [8, 40]},
        {"name": "receptive_layers", "type": "range", "bounds": [0, 1]},
        {"name": "lr", "type": "range", "bounds": [1e-6, 0.01], "log_scale": True},
        {"name": "dropout", "type": "range", "bounds": [0.1, 0.5]},  
        {"name": "fc_out", "type": "range", "bounds": [6, 32], "log_scale": True},
        {"name": "weight_decay", "type": "range", "bounds": [1e-4, 0.1], "log_scale": True}
    ],
    evaluation_function=train_evaluate,
    objective_name='accuracy',
    total_trials=300)

[INFO 08-07 23:26:17] ax.service.utils.dispatch: Using Bayesian Optimization generation strategy. Iterations after 9 will take longer to generate due to model-fitting.
[INFO 08-07 23:26:17] ax.service.managed_loop: Started full optimization with 300 steps.
[INFO 08-07 23:26:17] ax.service.managed_loop: Running optimization trial 1...


{'batch_size': 42, 'filter1': 110, 'filter2': 6, 'main_kernel': 31, 'receptive_layers': 1, 'lr': 1.4045906665597491e-06, 'dropout': 0.3521549940109253, 'fc_out': 7, 'weight_decay': 0.06590757532744057}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 110, 4, 128]          13,750
              ReLU-2          [-1, 110, 4, 128]               0
            Conv2d-3            [-1, 6, 4, 128]           1,986
              ReLU-4            [-1, 6, 4, 128]               0
            Conv2d-5            [-1, 6, 4, 128]             114
              ReLU-6            [-1, 6, 4, 128]               0
         MaxPool2d-7             [-1, 6, 4, 64]               0
           Dropout-8                 [-1, 1536]               0
            Linear-9                    [-1, 7]          10,759
           Linear-10                    [-1, 6]              48
          Softmax-11         

[INFO 08-07 23:29:05] ax.service.managed_loop: Running optimization trial 2...


{'batch_size': 21, 'filter1': 15, 'filter2': 12, 'main_kernel': 8, 'receptive_layers': 1, 'lr': 0.0008826855691643706, 'dropout': 0.41681637763977053, 'fc_out': 28, 'weight_decay': 0.000325420448190594}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 15, 4, 128]             495
              ReLU-2           [-1, 15, 4, 128]               0
            Conv2d-3           [-1, 12, 4, 128]             552
              ReLU-4           [-1, 12, 4, 128]               0
            Conv2d-5           [-1, 12, 4, 128]             444
              ReLU-6           [-1, 12, 4, 128]               0
         MaxPool2d-7            [-1, 12, 4, 64]               0
           Dropout-8                 [-1, 3072]               0
            Linear-9                   [-1, 28]          86,044
           Linear-10                    [-1, 6]             174
          Softmax-11        

[INFO 08-07 23:31:55] ax.service.managed_loop: Running optimization trial 3...


...

{'batch_size': 169, 'filter1': 69, 'filter2': 17, 'main_kernel': 32, 'receptive_layers': 1, 'lr': 1.9997790880445e-06, 'dropout': 0.32393388055570704, 'fc_out': 19, 'weight_decay': 0.013953169981186002}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 69, 4, 128]           8,901
              ReLU-2           [-1, 69, 4, 128]               0
            Conv2d-3           [-1, 17, 4, 128]           3,536
              ReLU-4           [-1, 17, 4, 128]               0
            Conv2d-5           [-1, 17, 4, 128]             884
              ReLU-6           [-1, 17, 4, 128]               0
         MaxPool2d-7            [-1, 17, 4, 64]               0
           Dropout-8                 [-1, 4352]               0
            Linear-9                   [-1, 19]          82,707
           Linear-10                    [-1, 6]             120
          Softmax-11        

[INFO 08-08 02:27:50] ax.service.managed_loop: Running optimization trial 64...


{'batch_size': 47, 'filter1': 94, 'filter2': 6, 'main_kernel': 31, 'receptive_layers': 1, 'lr': 1.3925546320234054e-06, 'dropout': 0.34909488036217795, 'fc_out': 9, 'weight_decay': 0.03957175620141483}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 94, 4, 128]          11,750
              ReLU-2           [-1, 94, 4, 128]               0
            Conv2d-3            [-1, 6, 4, 128]           1,698
              ReLU-4            [-1, 6, 4, 128]               0
            Conv2d-5            [-1, 6, 4, 128]             114
              ReLU-6            [-1, 6, 4, 128]               0
         MaxPool2d-7             [-1, 6, 4, 64]               0
           Dropout-8                 [-1, 1536]               0
            Linear-9                    [-1, 9]          13,833
           Linear-10                    [-1, 6]              60
          Softmax-11         

[INFO 08-08 02:30:39] ax.service.managed_loop: Running optimization trial 65...


{'batch_size': 14, 'filter1': 35, 'filter2': 8, 'main_kernel': 13, 'receptive_layers': 1, 'lr': 1.8652510411725962e-06, 'dropout': 0.35065315461170066, 'fc_out': 31, 'weight_decay': 0.008847086192454456}
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 35, 4, 128]           1,855
              ReLU-2           [-1, 35, 4, 128]               0
            Conv2d-3            [-1, 8, 4, 128]             848
              ReLU-4            [-1, 8, 4, 128]               0
            Conv2d-5            [-1, 8, 4, 128]             200
              ReLU-6            [-1, 8, 4, 128]               0
         MaxPool2d-7             [-1, 8, 4, 64]               0
           Dropout-8                 [-1, 2048]               0
            Linear-9                   [-1, 31]          63,519
           Linear-10                    [-1, 6]             192
          Softmax-11       

In [ ]:
best_params

In [ ]:
means, covariances = values
means, covariances

In [ ]:
render(interact_contour(model=model, metric_name='accuracy'))

## Check to see if model accuracy improves as we identify better hyperparameters

In [ ]:
# `plot_single_method` expects a 2-d array of means, because it expects to average means from multiple 
# optimization runs, so we wrap out best objectives array in another array.
best_objectives = np.array([[trial.objective_mean*100 for trial in experiment.trials.values()]])
best_objective_plot = optimization_trace_single_method(
    y=np.maximum.accumulate(best_objectives, axis=1),
    title="Model performance vs. # of iterations",
    ylabel="Classification Accuracy, %",
)
render(best_objective_plot)